# Fine-tune a pretrained model

There are significant benefits to using a pretrained model. It reduces computational costs, allows you to use state-of-the-art models without having to train one from scratch.
Hugging Face *Transformers* provides access to  thousands of pretrained models for a wide range of tasks. When you use a pretrained model, you train it on a dataset specific to your task. This is known as fine-tuning, an incredibly powerful training techinique. In this tutorial, you will fine-tune a pretrained model with a deep learning framework of your choice:

* Fine-tune a pretrained model with Transformers *Trainer*.
* Fine-tune a pretrained model in Tensorflow with Keras.
* Fine-tune a pretrained model in native PyTorch.

## Fine-tune a pretrained model with Transformers Trainer

### Prepare dataset
Before you can fine-tune a pretrained model, load a dataset and prepare it for training. Here let's use *Yelp Reviews* dataset as an example.

In [8]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")#this dataset can be loaded from hugging face directly, https://huggingface.co/docs/datasets/index
dataset["train"][100]#an example in train set

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to C:\Users\Opal.Yang\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

Note: Datasets can be loaded from local files stored on your computer too (csv/json/parquet/txt). The *load_dataset()* function can load each of these file types. For example:

In [ ]:
from datasets import load_dataset

#load a single csv file
dataset = load_dataset("csv", data_files="path_to_my_file.csv")

#or load more than one csv file
dataset = load_dataset("csv", data_files=["my_file_1.csv", "my_file_2.csv", "my_file_3.csv"])

#you can also map the training and testing splits to specific CSV files
dataset = load_dataset("csv", data_files={"train": ["my_file_1.csv", "my_file_2.csv"], "test":["my_file_3.csv"]})

##more details: https://huggingface.co/docs/datasets/loading

### Tokenize dataset
You need a tokenizer to process the text and include a padding and truncation strategy to handle any variable sequence lengths. To process your dataset in one step, use Dataset map function to apply a preprocessing function over the entire dataset:

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
#Here we are using bert-base-cased as our pretrained model because it is a state-of-the-art model for handling classification tasks
#Depending on the downstream tasks we have, we can choose different pretrained model
#For more details, please refer to Hugging Face model card

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
#Setting batched=True enables us to tokenize several sentences simultaneously

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

  0%|          | 0/650 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

### Trainer
*Transformers* provides a *Trainer* class optimized for training models, making it easier to start training without manually writing your own training loop. The Trainer API supports a wide range of training options and features such as logging, gradient accumulation, and mixed precision. https://huggingface.co/docs/transformers/v4.21.0/en/main_classes/trainer#transformers.Trainer

Let's start by loading your model and specify the number of expected labels. From the Yelp Review dataset card, you know there are 5 labels:

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Note: You will see a warning about some of the pretrained weights not being used and some weights being randomly initialized. Don't worry, this is completely normal! The pretrained head of the BERT model is discarded, and replaced with a randomly initialized classification head. You will fine-tune this new model head on your sequence classification task, transferring the knowledge of the pretrained model to it.

#### Training hyperparameters
Next, create a *TrainingArguments* class which contains all the hyperparameters you can tune as well as flags for activating different training options. For this tutorial you can start with the default training hyperparameters, but feel free to experiment with these to find your optimal settings.
https://huggingface.co/docs/transformers/v4.21.0/en/main_classes/trainer#transformers.TrainingArguments

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "test_trainer"#specify where to save the checkpoints
)


#### Metrics
Trainer does not automatically evaluate model performance during training. You will need to pass Trainer a function to compute and report metrics. The Datasets library provides a simple accuracy function you can load with the load_metrics (https://huggingface.co/docs/datasets/metrics).

In [12]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

Call *compute* on *metric* to calculate the accuracy of your predictions. Before passing your predictions to *compute*, you need to convert the predictions to logits(all Transformers models return logits).

In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

If you'd like to monitor your evaluation metrics during fine-tuning, specify the *evaluation_strategy* parameter in your training arguments to report the evaluation metric at the end of each epoch:


In [ ]:
# training_args = TrainingArguments(
#     output_dir="test_trainer",
#     evaluation_strategy="epoch"
# )

#### Trainer
Create a Trainer object with your model, training arguments, training and testing datasets, and evaluation function:

In [15]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
)

trainer.train()#fine-tune your model by calling train()

## Fine-tune a pretrained model in Tensorflow with Keras

### Convert dataset to Tensorflow format
The *DefaultDataCollator* assembles tensors into a batch for the model to train on. Make sure you specify *return_tensors* to return Tensorflow tensors:

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")
#Trainer uses a DataCollatorWithPadding by default so you don't need to
#explictly specify a data collator.

In [ ]:
#convert the tokenized datasets to Tensorflow with the to_tf_dataset method
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8
)
tf_validation_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8
)

### Compile and fit
Load a Tensorflow model with the expected number of labels, then compile and fine-tune your model with *fits* as you would with any other Keras model.

In [ ]:
import tensorflow as tf 
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy()
)

model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

## Fine-tune a pretrained model in native PyTorch
For users who prefer to write their own training loop, you can also fine-tune a Transformers model in native PyTorch.

#### Prepare dataset

In [ ]:
#remove column text from tokenized_datasets because the model does
#not accept raw text as an input:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

In [ ]:
#rename the label column to labels because the model expects the argument to be named labels
tokenized_datasets = tokenized_datasets.rename_column("label","labels")

In [ ]:
#set the format of the dataset to return PyTorch tensors instead of lists
tokenized_datasets.set_format("torch")

#### Data loader
Create a DataLoader for your training and test datasets so you can iterate over batches of data:

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=8)
eval_dataloader = DataLoader(tokenized_datasets["test"], batch_size=8)

#### Load model
Load your model with the number of expected labels:

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

#### Specify optimizer and learning rate scheduler
Create an optimizer and learning rate scheduler to fine-tune the model:

In [ ]:
from torch.optim import AdamW
from transformers import get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs=3
num_training_steps = num_epochs*len(train_dataloader)
lr_scheduler=get_scheduler(
    name="linear",
    optimizer=optimizer, 
    num_warmup_steps=0, 
    num_training_steps=num_training_steps
)

In [ ]:
#specify device to GPU if you have access to one
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

#### Training loop
To keep track of your training progress, use the tqdm library to add a progress bar over the number of training steps:


In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

#### Metrics
Just like how you need to add an evaluation function to Trainer, you need to do the same when you write your own training loop. But instead of calculating and reporting the metric at the end of each epoch, this time you will accumulate all the batches with add_batch and calculate the metric at the very end.

In [ ]:
metric = load_metric("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

## Additional Resources
For more fine-tuning examples, refer to:
https://github.com/huggingface/transformers/tree/main/examples
https://huggingface.co/docs/transformers/notebooks